In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# Attempt 1

# NAME of the applicant convert to data points

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN'], axis=1, inplace=True)

In [3]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

# Determine which values to replace if counts are less than ...?
replace_app = list(app_type_counts[app_type_counts < 500].index)

# Replace in dataframe
for app in replace_app:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat


['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [6]:
# Create a OneHotEncoder instancea
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True,right_index=True)
application_df = application_df.drop(columns = application_cat)
application_df.head()

C:\Users\ericn\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_1 DAY RANCH RESCUE AND RURAL OKLAHOMA ANIMAL RESOURCE INC,NAME_100 BLACK MEN OF AMERICA,NAME_100 BLACK MEN OF MEMPHIS INC,NAME_100 BLACK MEN OF WEST GEORGIA INC,NAME_1150 WEBSTER STREET INC,NAME_116TH CAVALRY REGIMENT CHAPTER OF THE US CAVALRY & ARMOR ASSOCIATION,NAME_13TH BOMB SQUADRON ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'],1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\ericn\AppData\Local\Temp\ipykernel_3648\1757507527.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df.drop(['IS_SUCCESSFUL'],1).values


In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                1568960   
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1,571,421
Trainable params: 1,571,421
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [10]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq="epoch",
                              period=5)

In [12]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.4852 - accuracy: 0.7657
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.1290 - accuracy: 0.9536
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.0968 - accuracy: 0.9612
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.0933 - accuracy: 0.9620
Epoch 5/100
792/804 [============================>.] - ETA: 0s - loss: 0.0922 - accuracy: 0.9624
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.0926 - accuracy: 0.9623
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0909 - accuracy: 0.9623
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0904 - accuracy: 0.9626
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0897 - accuracy: 0.9630
Epoch 9/100
804/804 [==============================] -

804/804 [==============================] - ETA: 0s - loss: 0.0760 - accuracy: 0.9686
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 4s 4ms/step - loss: 0.0760 - accuracy: 0.9686
Epoch 66/100
804/804 [==============================] - 4s 4ms/step - loss: 0.0766 - accuracy: 0.9686
Epoch 67/100
804/804 [==============================] - 3s 4ms/step - loss: 0.0760 - accuracy: 0.9685
Epoch 68/100
804/804 [==============================] - 3s 4ms/step - loss: 0.0765 - accuracy: 0.9689
Epoch 69/100
804/804 [==============================] - 3s 4ms/step - loss: 0.0761 - accuracy: 0.9688
Epoch 70/100
799/804 [============================>.] - ETA: 0s - loss: 0.0764 - accuracy: 0.9686
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.0762 - accuracy: 0.9687
Epoch 71/100
804/804 [==============================] - 3s 4ms/step - loss: 0.0759 - accuracy: 0.9685
Epoch 72/100
804/804 

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.7687 - accuracy: 0.5879 - 703ms/epoch - 3ms/step
Loss: 0.7686880230903625, Accuracy: 0.5878717303276062


In [14]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation.h5")

# Attempt 2

# Adding Additional neurons to hidden layers and additional layers are added

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               1961200   
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 30)                1530      
                                                                 
 dense_6 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1,967,811
Trainable params: 1,967,811
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq="epoch",
                              period=5)

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 4s 5ms/step - loss: 0.4794 - accuracy: 0.7716
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1423 - accuracy: 0.9472
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0978 - accuracy: 0.9616
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0935 - accuracy: 0.9621
Epoch 5/100
799/804 [============================>.] - ETA: 0s - loss: 0.0927 - accuracy: 0.9612
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.0927 - accuracy: 0.9612
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0919 - accuracy: 0.9616
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0912 - accuracy: 0.9621
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0899 - accuracy: 0.9627
Epoch 9/100
804/804 [==============================] -

794/804 [============================>.] - ETA: 0s - loss: 0.0761 - accuracy: 0.9690
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.0760 - accuracy: 0.9691
Epoch 66/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0760 - accuracy: 0.9692
Epoch 67/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0764 - accuracy: 0.9694
Epoch 68/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0762 - accuracy: 0.9692
Epoch 69/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0763 - accuracy: 0.9692
Epoch 70/100
804/804 [==============================] - ETA: 0s - loss: 0.0759 - accuracy: 0.9689
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.0759 - accuracy: 0.9689
Epoch 71/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0759 - accuracy: 0.9690
Epoch 72/100
804/804 

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6985 - accuracy: 0.7395 - 433ms/epoch - 2ms/step
Loss: 0.6985084414482117, Accuracy: 0.7394751906394958


In [21]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation.h5")

# Attempt 3

# Using different activation functions for the hidden layers.(relu to sigmoid)

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               1961200   
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 30)                1530      
                                                                 
 dense_10 (Dense)            (None, 1)                 31        
                                                                 
Total params: 1,967,811
Trainable params: 1,967,811
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq="epoch",
                              period=5)

In [26]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 5s 5ms/step - loss: 0.4863 - accuracy: 0.7734
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1431 - accuracy: 0.9525
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1028 - accuracy: 0.9616
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0953 - accuracy: 0.9624
Epoch 5/100
798/804 [============================>.] - ETA: 0s - loss: 0.0917 - accuracy: 0.9629
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.0919 - accuracy: 0.9628
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0899 - accuracy: 0.9635
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0892 - accuracy: 0.9633
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0881 - accuracy: 0.9634
Epoch 9/100
804/804 [==============================] -

803/804 [============================>.] - ETA: 0s - loss: 0.0785 - accuracy: 0.9682
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.0784 - accuracy: 0.9683
Epoch 66/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0783 - accuracy: 0.9679
Epoch 67/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0780 - accuracy: 0.9679
Epoch 68/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0784 - accuracy: 0.9680
Epoch 69/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0778 - accuracy: 0.9684
Epoch 70/100
796/804 [============================>.] - ETA: 0s - loss: 0.0779 - accuracy: 0.9684
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.0781 - accuracy: 0.9683
Epoch 71/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0780 - accuracy: 0.9686
Epoch 72/100
804/804 

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5144 - accuracy: 0.7534 - 1s/epoch - 5ms/step
Loss: 0.5144386887550354, Accuracy: 0.7533527612686157


In [28]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation.h5")